Updated April 2024, works for regular estim protocols (not trains) and old/new visual stim logs (circles and natural scenes).

In [1]:
import io
import sys
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import copy
import time

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp

In [4]:
%matplotlib notebook

## Load data

In [5]:
# rec_folder = r'F:\testing\mouse000000\old_test_2023-09-07_14-07-36\experiment1\recording1' # old log style
rec_folder = r'V:\mouse730911\aw_stim_train_2024-05-03_10-33-23\experiment1\recording1'
exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)

Experiment type: electrical stimulation


In [6]:
sync_data = exp._load_sync_dataset()

# Estim log

In [7]:
esweep_rising = sync_data.get_edges(keys=('estim_sweep',), kind='rising', units='seconds')
esweep_falling = sync_data.get_edges(keys=('estim_sweep',), kind='falling', units='seconds')
esync_rising = sync_data.get_edges(keys=('estim_sync',), kind='rising', units='seconds')
esync_falling = sync_data.get_edges(keys=('estim_sync',), kind='falling', units='seconds')

In [8]:
total_trials = len(esync_rising)
print('Total e-stim trials: %d' % total_trials)
num_sweeps = len(esweep_rising)
print('Number of e-stim sweeps: %d' % num_sweeps)
trials_per_sweep = int(total_trials / num_sweeps)
print('Trials per sweep: %d' % trials_per_sweep)

Total e-stim trials: 150
Number of e-stim sweeps: 2
Trials per sweep: 75


#### Read multichannel systems stim file

In [9]:
print(exp.stim_instruction_files['electrical'])

['V:\\mouse730911\\aw_stim_train_2024-05-03_10-33-23\\experiment1\\recording1\\stimulation_protocol_1pulse_50tr_25uA_6.5-7.5_ISI.txt', 'V:\\mouse730911\\aw_stim_train_2024-05-03_10-33-23\\experiment1\\recording1\\stimulation_protocol_2pulse_50tr_25uA_6.5-7.5_ISI.txt']


In [19]:
with open(exp.stim_instruction_files['electrical'][0]) as file:
    estim_txt = file.read() # open estim file and read in text
estim = estim_txt.split('channel:') # split and read txt file
d = {}
for item in estim:
    if 'value' in item:
        key = 'channel'+str(item[1])
        d[key]=item.split('\n')
channel1 = d['channel1'][1:]
ch1_df = pd.read_csv(io.StringIO('\n'.join(channel1)), delim_whitespace=True)

In [21]:
ch1_df[:10]

,value,time
0,-25,200.0
1,25,200.0
2,0,200000.0
3,0,6801455.0
4,-25,200.0
5,25,200.0
6,0,200000.0
7,0,7201099.0
8,-25,200.0
9,25,200.0


In [16]:
list_stim_type = []
list_stim_amp = []
list_stim_dur = []
for i in range(1, len(ch1_df)):
    if ch1_df.value.iloc[i-1] < 0 and ch1_df.value.iloc[i] > 0:
        list_stim_type.append('biphasic')
        list_stim_amp.append(ch1_df.value.iloc[i])
        list_stim_dur.append(ch1_df.time.iloc[i-1] + ch1_df.time.iloc[i])
estim_log = pd.DataFrame({
    'stim_type': list_stim_type,
    'parameter': list_stim_amp,
    'duration': np.array(list_stim_dur) * 1e-6
})

In [17]:
print(len(estim_log))
estim_log.head()

100


,stim_type,parameter,duration
0,biphasic,25,0.0004
1,biphasic,25,0.0004
2,biphasic,25,0.0004
3,biphasic,25,0.0004
4,biphasic,25,0.0004


In [22]:
if len(estim_log) == trials_per_sweep:
    estim_log = pd.concat([estim_log] * num_sweeps, axis='index', ignore_index=True)
else:
    print('estim_log does not match total_trials / num_sweeps')

estim_log does not match total_trials / num_sweeps


In [ ]:
estim_log['onset'] = esync_rising
estim_log['offset'] = esync_falling

In [ ]:
estim_log['sweep'] = np.zeros(len(estim_log), dtype=int) - 1
for i in range(len(esweep_rising)):
    # print('Sweep %d: %f to %f' % (i, sweep_start, sweep_end))
    estim_log.loc[(estim_log['onset'] >= esweep_rising[i]-5) & (estim_log['onset'] <= esweep_falling[i]+2), 'sweep'] = i

In [ ]:
estim_log.head()

# Sensory stim

#### Set parameters and load sync file

In [6]:
VISUAL_STIMULI = ['fullscreen', 'circle', 'natural_scene']

In [7]:
sweep_rising = sync_data.get_edges(keys=('behavior_sweep',), kind='rising', units='seconds')
sweep_falling = sync_data.get_edges(keys=('behavior_sweep',), kind='falling', units='seconds')

vsync_rising = sync_data.get_edges(keys=('behavior_vsync',), kind='rising', units='seconds')
vsync_falling = sync_data.get_edges(keys=('behavior_vsync',), kind='falling', units='seconds')

pd_rising = sync_data.get_edges(keys=('photodiode',), kind='rising', units='seconds')
pd_falling = sync_data.get_edges(keys=('photodiode',), kind='falling', units='seconds')

if len(sweep_rising) == 0:
    between_sweep_inds = np.argwhere(np.diff(vsync_rising) > 10.).flatten()
    sweep_rising = np.concatenate(([vsync_rising[0]-1.], vsync_rising[between_sweep_inds+1]-1.))
    sweep_falling = np.concatenate((vsync_falling[between_sweep_inds]+1., [vsync_falling[-1]+1.]))

#### Load experiment log file

In [8]:
explogfile = pd.read_csv(exp.stim_instruction_files['sensory'][0])
explogfile.head()

,stim_type,parameter,ISI,duration,imtrial.thisRepN,imtrial.thisTrialN,imtrial.thisN,imtrial.thisIndex,thisRow.t,notes,trialStim.started,trialStim.stopped,Unnamed: 12
0,natural_scene,97,2.412221,1.0,0,0,0,0,NaN,NaN,5.129218,6.143125,NaN
1,natural_scene,59,2.274849,0.2,0,1,1,1,NaN,NaN,8.514028,8.728835,NaN
2,natural_scene,107,2.489749,0.2,0,2,2,2,NaN,NaN,10.973845,11.180777,NaN
3,natural_scene,60,2.895042,1.0,0,3,3,3,NaN,NaN,13.668874,14.683870,NaN
4,natural_scene,71,2.665113,0.5,0,4,4,4,NaN,NaN,17.547715,18.053506,NaN


In [9]:
if 'display_stim' in list(explogfile.columns):
    print('This is the old style log')
    explogfile = explogfile.drop(labels='time', axis=1)

    ## Remove gray screen from log ##
    list_stim_type = []
    list_stim_param = []
    for i in range(1, len(explogfile)):
        if explogfile.stim_type.iloc[i-1] == 'gray_screen' and explogfile.stim_type.iloc[i] in VISUAL_STIMULI:
            list_stim_type.append(explogfile.stim_type.iloc[i])
            list_stim_param.append(explogfile.stim_parameter.iloc[i]) # to make sure order matches stim table
    stim_log = pd.DataFrame({
        'stim_type': list_stim_type,
        'parameter': list_stim_param,
    })
    
elif 'ISI' in list(explogfile.columns):
    print('This is the new style log')
    
    stim_log = explogfile.drop(
        labels=[
            'imtrial.thisRepN','imtrial.thisTrialN','imtrial.thisN','imtrial.thisIndex','thisRow.t','notes',
            'trialStim.started','trialStim.stopped','Unnamed: 12'
        ], axis=1
    )
    
else:
    print("Uh oh, tbd_eeg does not understand this log file.")

This is the new style log


#### Add photodiode times as onsets/offsets (if available)

In [10]:
if pd_rising.shape[0] == stim_log.shape[0]:
    stim_log['onset'] = pd_rising
    if pd_falling.shape[0] == stim_log.shape[0]:
        stim_log['offset'] = pd_falling
    else:
        print('Number of photodiode offsets does not match number of stimuli; estimating offsets.')
        stim_log['offset'] = stim_log['onset'] + stim_log['duration']
elif vsync_rising.shape[0] == stim_log.shape[0]:
    print('Number of photodiode onsets does not match number of stimuli; using vsync times instead (these are typically 50 ms early).')
    stim_log['onset'] = vsync_rising
    stim_log['offset'] = vsync_falling
else:
    print('Number of photodiode and vsync pulses do not match number of stimuli; manual stim_log creation is needed.')

#### Add duration (for old log) and sweep number

In [11]:
if 'duration' not in list(explogfile.columns):
    stim_log['duration'] = stim_log['offset'] - stim_log['onset']
else:
    print('New log already has duration.')

New log already has duration.


In [12]:
stim_log['sweep'] = np.zeros(len(stim_log), dtype=int) - 1
for i in range(len(sweep_rising)):
    # print('Sweep %d: %f to %f' % (i, sweep_start, sweep_end))
    stim_log.loc[(stim_log['onset'] > sweep_rising[i]) & (stim_log['onset'] < sweep_falling[i]), 'sweep'] = i

#### Check it

In [13]:
stim_log.head()

,stim_type,parameter,ISI,duration,onset,offset,sweep
0,natural_scene,97,2.412221,1.0,18.05894,19.06041,0
1,natural_scene,59,2.274849,0.2,21.44512,21.64591,0
2,natural_scene,107,2.489749,0.2,23.89716,24.09797,0
3,natural_scene,60,2.895042,1.0,26.59942,27.60087,0
4,natural_scene,71,2.665113,0.5,30.46932,30.97036,0


# Put them all together and save

In [ ]:
all_logs_list = [estim_log, stim_log] # estim is called estim_log and visual is called stim_log
all_stim_log = pd.concat(all_logs_list, axis=0, sort=False).sort_values(by='onset', axis=0)

In [ ]:
all_stim_log.to_csv(exp.stimulus_log_file, index=False)